# Covid-19 Data Science Project

Eine Datenanalyse zu Covid-19 Daten. Ziel dieser Datenanalyse ist es ein Modell zu entwickeln, dass die zukünftigen Fallzahlen anhand verschiedener Input-Variablen wie dem Standort, den aktuellen Maßnahmen (Lockdown) und den vorherigen Fallzahlen vorhersagen kann. Zur Analyse werden Regressionsmodelle verwendet. Anschließend wird ein Deep Learning Algorithmus (neuronales Netz) angewendet.

In [30]:
import io
import urllib
import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from progressbar import ProgressBar

Historische Daten vom RKI zum Bundesland Baden-Württemberg.

Spaltenbeschreibung: https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0

Weitere Beschreibungen: https://www.bbsr.bund.de/BBSR/DE/forschung/raumbeobachtung/InteraktiveAnwendungen/corona-dashboard/corona-dashboard_einstieg.html

Berechnung der 7-Tage-Inzidenz mittels Meldedatum: https://lua.rlp.de/de/presse/detail/news/News/detail/corona-hinweise-zur-berechnung-der-7-tage-inzidenz/

In [31]:
# # Datendownload
# url_cases_rki = "https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv?where=IdBundesland%20%3E%3D%208%20AND%20IdBundesland%20%3C%3D%208"
# filename_cases = "./RKI_daily.csv"

# urllib.request.urlretrieve(url_cases_rki, filename_cases)

In [32]:
data_all = pd.read_csv("./RKI_daily.csv")

In [33]:
# data_all["Altersgruppe"].unique()

data_all = data_all.sort_values(by=['Landkreis','Meldedatum'])\
            .drop(["ObjectId", "IdBundesland", "Bundesland", "Altersgruppe2", "AnzahlTodesfall",\
                    "NeuerFall", "NeuerTodesfall", "Refdatum", "NeuGenesen", "IstErkrankungsbeginn"], axis=1)

mask_age = (data_all["Altersgruppe"] == "A60-A79") | (data_all["Altersgruppe"] == "A80+")
data_all["AnzahlFall>59"] = 0
data_all["AnzahlFall>59"][mask_age] = data_all["AnzahlFall"]
# Alternativ new column mit numpy.where erstellen

data_all = pd.get_dummies(data_all, prefix=['Geschlecht'], columns=['Geschlecht'])
data_all = data_all.drop(["Geschlecht_unbekannt"], axis=1)

C:\Users\ba_wi\AppData\Local\Continuum\anaconda3\envs\Covid-19\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [34]:
anzahl_faelle = data_all["AnzahlFall"].sum()
data_all

,Landkreis,Altersgruppe,AnzahlFall,Meldedatum,IdLandkreis,Datenstand,AnzahlGenesen,AnzahlFall>59,Geschlecht_M,Geschlecht_W
160415,LK Alb-Donau-Kreis,A35-A59,1,2020/02/28 00:00:00+00,8425,"17.02.2021, 00:00 Uhr",1,0,1,0
160416,LK Alb-Donau-Kreis,A35-A59,1,2020/03/04 00:00:00+00,8425,"17.02.2021, 00:00 Uhr",1,0,1,0
160417,LK Alb-Donau-Kreis,A35-A59,1,2020/03/04 00:00:00+00,8425,"17.02.2021, 00:00 Uhr",1,0,1,0
161149,LK Alb-Donau-Kreis,A15-A34,1,2020/03/04 00:00:00+00,8425,"17.02.2021, 00:00 Uhr",1,0,1,0
160418,LK Alb-Donau-Kreis,A35-A59,1,2020/03/07 00:00:00+00,8425,"17.02.2021, 00:00 Uhr",1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
159898,SK Ulm,A35-A59,2,2021/02/16 00:00:00+00,8421,"17.02.2021, 00:00 Uhr",0,0,0,1
160543,SK Ulm,A60-A79,1,2021/02/16 00:00:00+00,8421,"17.02.2021, 00:00 Uhr",0,1,1,0
160716,SK Ulm,A60-A79,1,2021/02/16 00:00:00+00,8421,"17.02.2021, 00:00 Uhr",0,1,0,1
160876,SK Ulm,A80+,2,2021/02/16 00:00:00+00,8421,"17.02.2021, 00:00 Uhr",0,2,0,1


In [35]:
data_all["Geschlecht_M"] = data_all["Geschlecht_M"] * data_all["AnzahlFall"]
data_all["Geschlecht_W"] = data_all["Geschlecht_W"] * data_all["AnzahlFall"]

data_all.dtypes

Landkreis        object
Altersgruppe     object
AnzahlFall        int64
Meldedatum       object
IdLandkreis       int64
Datenstand       object
AnzahlGenesen     int64
AnzahlFall>59     int64
Geschlecht_M      int64
Geschlecht_W      int64
dtype: object

In [36]:
data_all["Meldedatum"] = data_all["Meldedatum"].str.slice(stop=10)
data_all["Meldedatum"] = pd.to_datetime(data_all["Meldedatum"], format='%Y/%m/%d') - pd.to_timedelta(7, unit='d')

In [37]:
landkreise_id = data_all[["Landkreis", "IdLandkreis"]].drop_duplicates()

In [38]:
data_all = data_all.groupby(['IdLandkreis', pd.Grouper(key='Meldedatum', freq='W-FRI')])['AnzahlFall', "AnzahlGenesen", "AnzahlFall>59", "Geschlecht_M", "Geschlecht_W"]\
       .sum()\
       .reset_index()\
       .sort_values(['IdLandkreis', 'Meldedatum'])

C:\Users\ba_wi\AppData\Local\Continuum\anaconda3\envs\Covid-19\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [39]:
data_all = data_all.merge(landkreise_id, left_on='IdLandkreis', right_on='IdLandkreis')
data_all

,IdLandkreis,Meldedatum,AnzahlFall,AnzahlGenesen,AnzahlFall>59,Geschlecht_M,Geschlecht_W,Landkreis
0,8111,2020-02-28,5,5,0,4,1,SK Stuttgart
1,8111,2020-03-06,78,78,3,41,37,SK Stuttgart
2,8111,2020-03-13,310,303,41,179,131,SK Stuttgart
3,8111,2020-03-20,275,261,65,139,136,SK Stuttgart
4,8111,2020-03-27,237,225,65,124,113,SK Stuttgart
...,...,...,...,...,...,...,...,...
2170,8437,2021-01-15,107,102,24,58,49,LK Sigmaringen
2171,8437,2021-01-22,130,127,20,63,64,LK Sigmaringen
2172,8437,2021-01-29,94,78,11,46,48,LK Sigmaringen
2173,8437,2021-02-05,44,7,9,19,25,LK Sigmaringen


In [40]:
print("Anzahl der Fälle stimmt noch überein? " + str(anzahl_faelle == data_all["AnzahlFall"].sum()))

Anzahl der Fälle stimmt noch überein? True


<br>
Bevölkerungsentwicklung der Stadt- und Landkreise in Baden-Württemberg
https://www.statistik-bw.de/BevoelkGebiet/Bevoelkerung/01035055.tab?R=LA

Bevölkerungsdichte der Stadt- und Landkreise in Baden-Württemberg
https://www.statistik-bw.de/BevoelkGebiet/Bevoelkerung/01515020.tab?R=LA

In [41]:
data_inhab = pd.read_csv("https://www.statistik-bw.de/BevoelkGebiet/Bevoelk_I_D_A_vj.csv",
                 encoding = "ISO-8859-1",
                 sep=";",
                 decimal=",",
                 skiprows=17)
data_inhab = data_inhab[(data_inhab["Amtlicher Gemeindeschlüssel (AGS)"]>1000) &
                        (data_inhab["Amtlicher Gemeindeschlüssel (AGS)"]<10000) &
                        # (data_inhab["Bevölkerung insgesamt"].str.isnumeric())==True &
                        (data_inhab["Stichtag"] == "30.09.2020")]
                         
# Aktuellster Stichtag und alles nur Schätzungen. Also keine Differenzierung notwendig

In [42]:
data_inhab

,Kürzel der Regionaleinheit,Amtlicher Gemeindeschlüssel (AGS),Regionalname,Stichtag,Bevölkerung insgesamt,Bevölkerung männlich,Bevölkerung weiblich,Deutsche zusammen,Deutsche männlich,Deutsche weiblich,Ausländer zusammen,Ausländer männlich,Ausländer weiblich
6989,KR,8111,Stadtkreis Stuttgart,30.09.2020,631688,315677,316011,474057,233142,240915,157631,82535,75096
6990,KR,8115,Landkreis Böblingen,30.09.2020,393609,195789,197820,319500,157258,162242,74109,38531,35578
6991,KR,8116,Landkreis Esslingen,30.09.2020,534525,266993,267532,439603,215137,224466,94922,51856,43066
6992,KR,8117,Landkreis Göppingen,30.09.2020,259076,129150,129926,214155,105076,109079,44921,24074,20847
6993,KR,8118,Landkreis Ludwigsburg,30.09.2020,545782,270971,274811,445920,218034,227886,99862,52937,46925
6994,KR,8119,Landkreis Rems-Murr-Kreis,30.09.2020,428007,211651,216356,358065,175040,183025,69942,36611,33331
6995,KR,8121,Stadtkreis Heilbronn,30.09.2020,126241,63617,62624,92994,45529,47465,33247,18088,15159
6996,KR,8125,Landkreis Heilbronn,30.09.2020,346652,174017,172635,294400,145993,148407,52252,28024,24228
6997,KR,8126,Landkreis Hohenlohekreis,30.09.2020,112964,56992,55972,99599,49667,49932,13365,7325,6040
6998,KR,8127,Landkreis Schwäbisch Hall,30.09.2020,197898,99252,98646,173652,86061,87591,24246,13191,11055


In [43]:
data_inhab_perkm = pd.read_csv("https://www.statistik-bw.de/BevoelkGebiet/Bevoelk_I_Flaeche_j.csv",
                 encoding = "ISO-8859-1",
                 sep=";",
                 decimal=",",
                 skiprows=18)

In [44]:
data_inhab_perkm = data_inhab_perkm[(data_inhab_perkm["Amtlicher Gemeindeschlüssel (AGS)"]>1000) &
                        (data_inhab_perkm["Amtlicher Gemeindeschlüssel (AGS)"]<10000) &
                        (data_inhab_perkm["Stichtag"] == "31.12.2019")]
# Aktuellster Stichtag

In [45]:
data_inhab_perkm

,Kürzel der Regionaleinheit,Amtlicher Gemeindeschlüssel (AGS),Postleitzahl,Regionalname,Stichtag,Bevölkerung insgesamt,Gemeindegebiet ha,Bevölkerungsdichte EW/km²
4665,KR,8111,X,Stadtkreis Stuttgart,31.12.2019,635911,20733,3067
4666,KR,8115,X,Landkreis Böblingen,31.12.2019,392807,61776,636
4667,KR,8116,X,Landkreis Esslingen,31.12.2019,535024,64128,834
4668,KR,8117,X,Landkreis Göppingen,31.12.2019,258145,64234,402
4669,KR,8118,X,Landkreis Ludwigsburg,31.12.2019,545423,68677,794
4670,KR,8119,X,Landkreis Rems-Murr-Kreis,31.12.2019,427248,85808,498
4671,KR,8121,X,Stadtkreis Heilbronn,31.12.2019,126592,9990,1267
4672,KR,8125,X,Landkreis Heilbronn,31.12.2019,344456,109991,313
4673,KR,8126,X,Landkreis Hohenlohekreis,31.12.2019,112655,77676,145
4674,KR,8127,X,Landkreis Schwäbisch Hall,31.12.2019,196761,148407,133


<br>
Altersstruktur der Stadt- und Landkreise in Ba-Wü
https://www.statistik-bw.de/BevoelkGebiet/Alter/98015200.tab?R=KR237

Download jeweils mittels des Amtlichen Gemeindeschlüssels (AGS)

In [46]:
pbar = ProgressBar()

AGS = ( pd.unique(data_inhab_perkm["Amtlicher Gemeindeschlüssel (AGS)"]) )%1000
data_age = pd.DataFrame(columns = ["Jahr", "Unter20", "Über65", "20bis65", "AGS"])

for ags in pbar(AGS):
    temp_data = pd.read_csv("https://www.statistik-bw.de/BevoelkGebiet/Alter/98015200.tab?R=KR" + str(ags) + "&form=csv",
                 encoding = "ISO-8859-1",
                 sep=";",
                 decimal=",",
                 names = ["Jahr", "Unter20", "Über65", "20bis65"],
                 usecols=[0,1,2,3],
                 skiprows = 28,
                 skipfooter = 20,
                    engine='python')
    temp_data["AGS"] = ags
    data_age = data_age.append(temp_data, ignore_index=True)

data_age["AGS"] = data_age["AGS"] + 8000

100% |########################################################################|


In [47]:
data_age

,Jahr,Unter20,Über65,20bis65,AGS
0,2020,109.680,113.716,421.919,8111
1,2021,109.892,114.218,423.461,8111
2,2020,78.538,80.109,236.644,8115
3,2021,78.881,81.539,236.339,8115
4,2020,101.883,111.712,325.572,8116
...,...,...,...,...,...
83,2021,40.311,50.212,126.459,8435
84,NaN,NaN,NaN,NaN,8436
85,NaN,NaN,NaN,NaN,8436
86,2020,25.482,27.600,78.546,8437


In [48]:
data_covid_regulations = pd.read_csv('Corona_Verordnungen.csv', header=None, sep='\n')
data_covid_regulations = data_covid_regulations[0].str.split(';', expand=True)
data_covid_regulations

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,Fassung,Anfangsdatum,Enddatum,Schulen geschlossen?,bis:,Hochschulen geschlossen?,Sportanlagen geschlossen?,Restaurants geschlossen?,Versammlungen im Öffentlichem Raum?,Private Versammlungen?,Maskenpflicht im ÖPVN?,Maskenpflicht im Einzelhandel?,Einzelhandel geschlossen?,Einschränkungen EH?,Ausgangssprerre?,Abstandsregelungen,Anmerkungen
1,1.0,17.03.2020,15.06.2020,1,19.04.2020,1,1,1,100 Personen,,0,0,1,,0,"1,5 m",
2,1.1,21.03.2020,15.06.2020,1,19.04.2020,1,1,1,3 Personen,5,0,0,1,,0,"1,5 m",Keine Fahrten in Risikogebiete
3,1.2,22.03.2020,15.06.2020,1,19.04.2020,1,1,1,1 Person,5,0,0,1,,0,"1,5 m",
4,1.3,29.03.2020,15.06.2020,1,19.04.2020,1,1,1,,5,0,0,1,,0,"1,5 m",Verstoß == Ordnun gswiedrigkeit // Zutrittskon...
5,1.4,10.04.2020,15.06.2020,1,19.04.2020,1,1,1,,5,0,0,1,,0,"1,5 m",
6,1.5,20.04.2020,15.06.2020,1,03.05.2020,1,1,1,,5,0,0,0,Alles unter 800qm,0,"1,5 m",
7,1.6,27.04.2020,15.06.2020,1,03.05.2020,1,1,1,,5,1,1,0,,0,"1,5 m",
8,1.6,04.05.2020,15.06.2020,0,,1,1,1,,5,1,1,0,,0,"1,5 m",
9,1.7,04.05.2020,15.06.2020,0,,1,1,1,,5,1,1,0,,0,"1,5 m",


In [49]:
data_age['Bevölkerung gesamt'] = data_age['Unter20'] + data_age['Über65'] + data_age['20bis65']
data_age['Ü65%'] = data_age['Über65']/ data_age['Bevölkerung gesamt'] 
data_age

data_inhab_perkm1 = data_inhab_perkm.drop(['Kürzel der Regionaleinheit', 'Postleitzahl', 'Regionalname', 'Stichtag', 'Bevölkerung insgesamt', 'Gemeindegebiet ha'], axis = 1)
#data_inhab_perkm1

data_inhab['Ausländer%'] = data_inhab['Ausländer zusammen'].astype(int) / data_inhab['Bevölkerung insgesamt'].astype(int)
data_inhab = data_inhab.drop(['Kürzel der Regionaleinheit', 'Regionalname', 'Stichtag', 'Bevölkerung insgesamt', 'Bevölkerung männlich',\
                            'Bevölkerung weiblich', 'Deutsche männlich', 'Deutsche weiblich',\
                            'Deutsche zusammen', 'Ausländer zusammen', 'Ausländer männlich', 'Ausländer weiblich'], axis = 1)

In [50]:
data_age

,Jahr,Unter20,Über65,20bis65,AGS,Bevölkerung gesamt,Ü65%
0,2020,109.680,113.716,421.919,8111,645.315,0.176218
1,2021,109.892,114.218,423.461,8111,647.571,0.176379
2,2020,78.538,80.109,236.644,8115,395.291,0.202658
3,2021,78.881,81.539,236.339,8115,396.759,0.205513
4,2020,101.883,111.712,325.572,8116,539.167,0.207194
...,...,...,...,...,...,...,...
83,2021,40.311,50.212,126.459,8435,216.982,0.231411
84,NaN,NaN,NaN,NaN,8436,NaN,NaN
85,NaN,NaN,NaN,NaN,8436,NaN,NaN
86,2020,25.482,27.600,78.546,8437,131.628,0.209682


In [51]:
data_all = data_all.merge(right= data_age, how = 'outer', right_on = 'AGS', left_on = 'IdLandkreis')
data_all2020 = data_all[(data_all["Meldedatum"].astype(str).str.contains(pat = '2020'))&
                        (data_all["Jahr"].astype(str).str.contains(pat = '2020'))]
data_all2021 = data_all[(data_all["Meldedatum"].astype(str).str.contains(pat = '2021'))&
                        (data_all["Jahr"].astype(str).str.contains(pat = '2021'))]
data_all = pd.concat([data_all2020, data_all2021])
data_all = data_all.drop(['Jahr', 'Unter20', 'Über65', '20bis65', 'AGS', 'Bevölkerung gesamt'], axis = 1)
data_all = data_all.merge(right = data_inhab, how = 'inner', right_on = 'Amtlicher Gemeindeschlüssel (AGS)', left_on = 'IdLandkreis')
data_all = data_all.drop(['Amtlicher Gemeindeschlüssel (AGS)'], axis = 1)
data_all = data_all.merge(right = data_inhab_perkm, how = 'inner', right_on = 'Amtlicher Gemeindeschlüssel (AGS)', left_on = 'IdLandkreis')
data_all = data_all.drop(['Amtlicher Gemeindeschlüssel (AGS)', 'Kürzel der Regionaleinheit', 'Postleitzahl', 'Regionalname', 'Stichtag', 'Bevölkerung insgesamt', 'Gemeindegebiet ha'], axis = 1)
data_all

,IdLandkreis,Meldedatum,AnzahlFall,AnzahlGenesen,AnzahlFall>59,Geschlecht_M,Geschlecht_W,Landkreis,Ü65%,Ausländer%,Bevölkerungsdichte EW/km²
0,8111,2020-02-28,5,5,0,4,1,SK Stuttgart,0.176218,0.249539,3067
1,8111,2020-03-06,78,78,3,41,37,SK Stuttgart,0.176218,0.249539,3067
2,8111,2020-03-13,310,303,41,179,131,SK Stuttgart,0.176218,0.249539,3067
3,8111,2020-03-20,275,261,65,139,136,SK Stuttgart,0.176218,0.249539,3067
4,8111,2020-03-27,237,225,65,124,113,SK Stuttgart,0.176218,0.249539,3067
...,...,...,...,...,...,...,...,...,...,...,...
2069,8437,2021-01-15,107,102,24,58,49,LK Sigmaringen,0.212952,0.113027,109
2070,8437,2021-01-22,130,127,20,63,64,LK Sigmaringen,0.212952,0.113027,109
2071,8437,2021-01-29,94,78,11,46,48,LK Sigmaringen,0.212952,0.113027,109
2072,8437,2021-02-05,44,7,9,19,25,LK Sigmaringen,0.212952,0.113027,109


<br>
<span style="color:red">
    Aufgabe:
</span>

- Bevölkerungszahlen den Kreisen zuordnen von data_inhab (unterschiedlich für 2020, für das jeweilige Datum die aktuellste Zahl nutzen)
- Bevölkerungsdichte den Kreisen zuordnen
- Der Anteil von Über65 Jährigen an Bevölkerung den Kreisen zuordnen
- Ausländeranteil berechnen und den Kreisen zuordnen

<br>

<br>
<span style="color:red">
    Aufgabe:
</span>

- Inzidenz-Werte berechnen mit den Bevölkerungszahlen (Oben ist ein Link zum "Wie")

<br>

<br>
<span style="color:red">
    Aufgabe:
</span>

- Für die Landkreise bzw. Ba-Wü allgemein Lockdown-Dummy erstellen
- 0 Kein Lockdown
- 0.5 Soft Lockdown
- 1 Hard Lockdown

?

<br>

<br>
<span style="color:red">
    Aufgabe:
</span>

Temperaturdurchschnitt zu Landkreis und Meldedatum abfragen:
https://github.com/panodata/dwdweather2

Dafür vermutlich Wetterstation den Landkreisen zuordnen.

Das ganze aber erst, nachdem wir 7-Tage-Inzidenzen nach Stichtagen haben. Brauchen die Daten nur zu diesen Stichtagen, nicht zu jedem Tag.
<br>

<br>
<span style="color:green">
    Beispiele:
</span>

- https://www.kaggle.com/docxian/covid-19-tracking-germany
- https://www.kaggle.com/mreverybody/covid19-germany-deutschland-rki-data
- Beispiele für Tabellen-Manipulation: https://datascience-enthusiast.com/R/pandas_datatable.html
- 